In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.signal as ss
import scipy.io
import scipy.stats as st
import os
import pandas as pd


def slice_signal(sig, tags, Fs):
    '''
    sig = single channel signal as a list or ndarray
    tags = tag file as a list or ndarray containing stimuli names (classes). Starting point (as a sample) of the stimuli corresponds to the index of the class in this array. tags and sig have the same length.
    Fs = integer, sampling frequency

    This function slices the signal based on the tag fule and returns a dictionary containing all signal slices for a particular stimuli class.
    '''
    result = {'1':[],'-1':[],'2':[]} #dict keys made specifically for this case
    ind = 0
    for item in tags:
        if item != [0]:
          result[str(item[0])].append(sig[ind-int(0.1*Fs): ind+int(0.6*Fs)])
        ind += 1
    return result#len(result['2']) powinno byc 60

#Ustawiam domyślne wielkości wykresów
plt.rcParams['figure.figsize'] = [20, 10]
plt.rcParams['figure.dpi'] = 100 
plt.rcParams.update({"axes.grid" : True, "grid.color": "black", "grid.linestyle": '--', "grid.linewidth": 0.5 })
#plt.subplots_adjust(top = 1, bottom = 0.3, wspace=1, hspace=2)

file = r'C:\Users\jezxi\Documents\Hackathon\P1_high1.mat'
print(file)
print(scipy.io.loadmat(file)['fs'])
print(len(scipy.io.loadmat(file)['y']),len(scipy.io.loadmat(file)['y'][0]))
print(len(scipy.io.loadmat(file)['trig']),len(scipy.io.loadmat(file)['trig'][0]))

data = scipy.io.loadmat(file)
import matplotlib.pyplot as plt
import numpy as np

channel_names = ['Fz', 'C3', 'Cz', 'CP1', 'CPz', 'CP2', 'Pz']

Fs = 256
T = len(data['y'][:,0])/Fs 
dt = 1/Fs
t = np.arange(0,T,dt)

# plt.title(channel_names[0])
# plt.plot(t, data['y'][:,0])

# os.listdir('C:\Users\jezxi\Documents\Hackathon')
path = r'C:\Users\jezxi\Documents\Hackathon\\'

num_channels = 8
channel_names = ['Fz', 'C3', 'Cz', 'C4', 'CP1', 'CPz', 'CP2', 'Pz']
file_name = []

P1 = {'1':[],'-1':[],'2':[]}
P2 = {'1':[],'-1':[],'2':[]}
for file in os.listdir(path):
    data = scipy.io.loadmat(path+file)
    Fs = data['fs']
    tags = data['trig']
    ''' 
    for i in range(num_channels):
        print(np.shape(data['y'][:,i]))
        psig = preprocess_signal(data['y'][:,i],Fs = Fs)
        slice_signal(psig, tags = data['trig'], Fs = Fs) #przygotowane dane z jednego kanału z jednego pliku
    ''' 
    psig = data['y']
    #psig = preprocess_signal(data['y'], Fs[0][0], num_channels=num_channels)
    if file[:2] == 'P1':
        res = slice_signal(psig, tags = data['trig'], Fs = Fs)#przygotowane dane z każdego kanału z jednego pliku
        P1 =  {x: P1.get(x, 0) + res.get(x, 0) for x in set(P1).union(res)}
    elif file[:2] == 'P2':
        res = slice_signal(psig, tags = data['trig'], Fs = Fs)#przygotowane dane z każdego kanału z jednego pliku
        P2 =  {x: P2.get(x, 0) + res.get(x, 0) for x in set(P2).union(res)}
    file_name = file_name + [file.split(".")[0]]*np.sum([len(val) for val in res.values()])*len(channel_names)



print(P1.keys(), len(P1['1']), len(P1['-1']), len(P1['2']))
print(P2.keys(), len(P2['1']), len(P2['-1']), len(P2['2']))
print(len(P1['1']), len(P1['1'][0]),len(P1['1'][0][0]))
print(len(P2['1']), len(P2['1'][0]),len(P2['1'][0][0]))

r = 1920
patients = np.array([np.vstack((P1['1'],P1['-1'],P1['2'])), np.vstack((P2['1'],P2['-1'],P2['2']))]) 
labels = np.zeros(r)
labels[:240] = -1
labels[240:1680] = 1
labels[1680:] = 2
# print(patients.shape)
# print(labels)
# len(labels)

knn_features = {"file_name": file_name, "patient": [], "channel": [], "class": [], "amplitude": [], "mean": [], "std": [], "med": [], 
                    "skewness": [], "kurtosis": [], "slope_change": [], "power": [], "energy": []}

p_id = 0

for p in patients:
    p_id += 1
    for frag_id in range(len(p)):
        for i in range(8):
            knn_features["patient"].append(p_id)
            knn_features["channel"].append(channel_names[i])
            knn_features["class"].append(labels[frag_id])
                        
            signal = p[frag_id][:,i]

            knn_features["mean"].append(np.mean(signal))
            knn_features["amplitude"].append(np.max(signal))
            knn_features["std"].append(np.std(signal, ddof=1))
            knn_features["med"].append(np.median(signal))
            knn_features["skewness"].append(st.skew(signal))
            knn_features["kurtosis"].append(st.kurtosis(signal))
            knn_features["energy"].append(np.sum(np.abs(signal)**2))
            T_loc = np.shape(signal)[0]/float(Fs)
            knn_features["power"].append(np.sum(np.abs(signal)**2)/T_loc)

            f = lambda x: (x >= np.median(signal)).astype(float)
            knn_features["slope_change"].append(np.sum(f(-np.diff(signal, prepend=1)[1:-1]*np.diff(signal)[1:])))
            
            # import IPython
            # IPython.embed()

# import IPython
# IPython.embed()



pd.DataFrame.from_dict(knn_features).to_csv("C:\\Users\\jezxi\\Documents\\hackathon_knn.csv")